In [23]:
import torch

In [24]:
word = 'helloworld'
sorted(word)

['d', 'e', 'h', 'l', 'l', 'l', 'o', 'o', 'r', 'w']

In [25]:
word2idx = {
    "d" : 0,
    "e": 1,
    "h": 2,
    "l": 3,
    "o": 4,
    "r": 5,
    "w": 6
}

idx2word = {}
for w in word2idx:
    idx2word[word2idx[w]] = w

def encode(seq):
    return [word2idx[w] for w in seq]

def decode(token):
    return [idx2word[i] for i in token]

def one_hot(idx, len = len(word2idx)):
    hots = []
    for i in range(len):
        if(i == idx):
            hots.append(1)
        else:
            hots.append(0)
    return hots

def batch_one_hot(seq):
    token = encode(seq)
    return [one_hot(i) for i in token]


In [33]:
input_size = 7
seq_len = 10
hidden_size = 7
batch_size = 1
num_layers = 2

In [40]:
input_seq = "helloworld"
output_seq ="worldhello"
x_data = batch_one_hot(input_seq)
y_data = batch_one_hot(output_seq)

inputs = torch.tensor(x_data).reshape(seq_len, batch_size, input_size).float()
labels = torch.tensor(y_data).reshape(-1, input_size).float()
print(inputs.shape)
print(labels.shape)

torch.Size([10, 1, 7])
torch.Size([10, 7])


In [47]:
class Model(torch.nn.Module):
    def __init__(self, input_size, hidden_size, batch_size, num_layers):
         super(Model, self).__init__()
         self.input_size = input_size
         self.hidden_size = hidden_size
         self.batch_size = batch_size
         self.num_layers = num_layers
         self.rnn = torch.nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=num_layers, bidirectional=False)

    def forward(self, inputs):
        init_hidden = torch.zeros(self.num_layers, self.batch_size, self.hidden_size)
        outputs, hidden = self.rnn(inputs, init_hidden)
        return outputs, hidden

net = Model(input_size, hidden_size, batch_size, num_layers)

In [51]:
outputs, hidden = net(inputs)
print("Output size(All hidden): ", outputs.shape) # seq, batch, hidden
print("Final Hidden size: ", hidden.shape) # num_layers, batch, hidden

Output size(All hidden):  torch.Size([10, 1, 7])
Final Hidden size:  torch.Size([2, 1, 7])


In [52]:
# hidden = hidden.view(2, 7)
hidden

tensor([[[ 0.5015, -0.3093, -0.2551,  0.2236,  0.0588, -0.1456, -0.2505]],

        [[-0.4099, -0.2838, -0.6060,  0.3740, -0.8044, -0.1748,  0.1702]]],
       grad_fn=<StackBackward0>)

In [36]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)

for epoch in range(15):
    optimizer.zero_grad()
    outputs = net(inputs)
    outputs = outputs.reshape(-1, hidden_size)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    _, idx = outputs.max(dim=1)
    idx = idx.data.numpy()
    print("Predicted string: ","".join(idx2word[x] for x in idx), end="")
    print(", Epoch [%d/15] loss=%.3f" % (epoch + 1, loss.item()))

Predicted string:  oooooooooo, Epoch [1/15] loss=1.935
Predicted string:  olllllllll, Epoch [2/15] loss=1.764
Predicted string:  wlllllllll, Epoch [3/15] loss=1.673
Predicted string:  wlllllllll, Epoch [4/15] loss=1.566
Predicted string:  wollllllll, Epoch [5/15] loss=1.444
Predicted string:  wooldhewlo, Epoch [6/15] loss=1.327
Predicted string:  wooldhewlo, Epoch [7/15] loss=1.259
Predicted string:  wooldhello, Epoch [8/15] loss=1.208
Predicted string:  wooldhello, Epoch [9/15] loss=1.147
Predicted string:  wooldhello, Epoch [10/15] loss=1.108
Predicted string:  wooldhello, Epoch [11/15] loss=1.071
Predicted string:  wooldhello, Epoch [12/15] loss=1.030
Predicted string:  wooldhello, Epoch [13/15] loss=0.985
Predicted string:  wooldhello, Epoch [14/15] loss=0.948
Predicted string:  wooodhello, Epoch [15/15] loss=0.921
